This notebook describes steps for installing the software and genotyping STRs using using HipSTR, GangSTR, ExpansionHunter, adVNTR and EnsembleTR.

# Obtain data

In [ ]:


# download other data resources from Zenodo
# reference genome
wget 
tar -xvf

# BAM files
wget 
tar -xvf

In [ ]:


# create output directory
mkdir -p output/{hipstr_output,advntr_output/advntr_dir,gangstr_output,eh_output,ensembletr_output}

tree output

# Installing tools

## HipSTR

In [ ]:


# HipSTR

# install these dependencies if you do no have them
# apt install g++ make zlib1g-dev libhts-dev libbz2-dev liblzma-dev

git clone https://github.com/gymrek-lab/HipSTR.git
cd HipSTR
make version && make

In [ ]:


./HipSTR --help
cd ..

## ExpansionHunter

In [ ]:


# ExpansionHunter

wget https://github.com/Illumina/ExpansionHunter/releases/download/v5.0.0/ExpansionHunter-v5.0.0-linux_x86_64.tar.gz

tar -xzvf ExpansionHunter-v5.0.0-linux_x86_64.tar.gz


In [ ]:


# The ExpansionHunter executable can be found in 
ExpansionHunter-v5.0.0-linux_x86_64/bin/ExpansionHunter --help


## GangSTR

In [ ]:
 
# GangSTR

mamba create -y --name gangstr
mamba activate gangstr
mamba config --add channels conda-forge
mamba config --add channels bioconda
mamba install -y -c conda-forge -c bioconda gangstr

In [ ]:
 
# this step maybe optional
# only run if you get this error
# GangSTR: error while loading shared libraries: libgsl.so.25: cannot open shared object file: No such file or directory
mamba install -y conda-forge::gsl

In [ ]:
 
GangSTR --help

mamba deactivate

## adVNTR

In [ ]:
 
# advntr
mamba create -y --name advntr
mamba activate advntr
mamba config --add channels conda-forge
mamba config --add channels bioconda
mamba install -y -c conda-forge -c bioconda advntr

In [ ]:
 
# this step is optional
# run if the step below returns
# WARNING (theano.configdefaults): install mkl with `conda install mkl-service`: No module named 'mkl'

mamba install -y mkl-service

In [ ]:
 

advntr 
mamba deactivate

## EnsembleTR

In [ ]:
 
# ensembleTR
git clone https://github.com/gymrek-lab/EnsembleTR.git
cd EnsembleTR
python3 setup.py install --user

In [ ]:
 
EnsembleTR --version
cd ..

## TRtools

In [ ]:
 
git clone https://github.com/gymrek-lab/TRTools.git
cd TRTools/
python3 -m pip install --upgrade pip
python3 -m pip install -e .

In [2]:
cd ..
./check_software.sh statSTR associaTR compareSTR dumpSTR mergeSTR prancSTR qcSTR simTR



### TRViz

In [ ]:
!pip install trviz

In [ ]:
!

# Genotyping STRs and preprocessing VCF files

## HipSTR

In [ ]:
 

# HipSTR

populations=("africa" "europe" "east_asia" "south_asia" "america")
reference_genome="references/reference_chroms.fa"

for population in "${populations[@]}"; do
    
    bam_files=$(cat str_resources/"${population}".txt | tr "\n" ",")

    HipSTR/HipSTR --bams $bam_files \
        --fasta $reference_genome \
        --regions str_resources/hipstr_reference.bed \
        --str-vcf output/hipstr_output/"${population}"_hipstr.vcf.gz \
        --log output/hipstr_output/"${population}"_strs.log \
        --viz-out output/hipstr_output/"${population}"_strs.viz.gz \
        --output-gls --output-pls --def-stutter-model
        
    tabix -p vcf output/hipstr_output/"${population}"_hipstr.vcf.gz
    
#     dumpSTR --vcf output/hipstr_output/"${population}"_hipstr.vcf.gz \
#         --out output/hipstr_output/"${population}_hipstr_filt" \
#         --vcftype 'hipstr'
    
#     bgzip output/hipstr_output/"${population}"_hipstr_filt.vcf
#     tabix -p vcf output/hipstr_output/"${population}"_hipstr_filt.vcf.gz

    ~/bcftools/bcftools sort -Oz output/hipstr_output/"${population}"_hipstr.vcf.gz -o output/hipstr_output/"${population}"_hipstr_sorted.vcf.gz
    tabix -p vcf output/hipstr_output/"${population}"_hipstr_sorted.vcf.gz
    
done

### Visualising HipSTR alignments

In [ ]:
 

# visualising alignments for the different superpopulations at chr17:51831668

## africa
HipSTR/VizAlnPdf output/hipstr_output/africa_strs.viz.gz chr17 51831668 HG02330 viz_HG02330 3

## europe
 HipSTR/VizAlnPdf output/hipstr_output/europe_strs.viz.gz chr17 51831668 HG00120 viz_HG00120 3

## america
HipSTR/VizAlnPdf output/hipstr_output/america_strs.viz.gz chr17 51831668 HG00742 viz_HG00742 3

## east asia
 HipSTR/VizAlnPdf output/hipstr_output/east_asia_strs.viz.gz chr17 51831668 HG00437 viz_HG00437 3

## south asia
HipSTR/VizAlnPdf output/hipstr_output/south_asia_strs.viz.gz chr17 51831668 HG02649 viz_HG02649 3

#### Africa - HG02330
![Africa](HG02330.png)


#### Europe - HG00120
![Europe](HG00120.png)


#### America - HG00742
![America](HG00742.png)


#### East Asia - HG00437
![East Asia](HG00437.png)


### South Asia - HG02649
![South Asia](HG02649.png)

## ExpansionHunter

In [ ]:
 
reference_genome="references/reference_chroms.fa"
output_dir=output/eh_output
male_list=str_resources/bams_male.txt
female_list=str_resources/bams_female.txt
eh_reference_strs=str_resources/eh_reference.json

# males
for sample_bam in $(cat $male_list); do
        
    sample_id=$(basename $sample_bam | cut -f1 -d".")        

    ExpansionHunter-v5.0.0-linux_x86_64/bin/ExpansionHunter \
            --threads 32 \
            --reads $sample_bam \
            --reference $reference_genome \
            --variant-catalog $eh_reference_strs \
            --output-prefix $output_dir/$sample_id \
            --sex male \
            --log-level trace

    # eh vcf files do not have contigs listed in the header
    bgzip output/eh_output/"${sample_id}".vcf
    tabix -p vcf output/eh_output/"${sample_id}".vcf.gz
    ~/bcftools/bcftools sort -Oz output/eh_output/"${sample_id}".vcf.gz -o output/eh_output/"${sample_id}"_sorted.vcf.gz
    tabix -p vcf output/eh_output/"${sample_id}"_sorted.vcf.gz

done


# females
for sample_bam in $(cat $female_list); do
    
    sample_id=$(basename $sample_bam | cut -f1 -d".")        

    ExpansionHunter-v5.0.0-linux_x86_64/bin/ExpansionHunter \
            --threads 32 \
            --reads $sample_bam \
            --reference $reference_genome \
            --variant-catalog $eh_reference_strs \
            --output-prefix $output_dir/$sample_id \
            --sex female \
            --log-level trace
                
    # eh vcf files do not have contigs listed in the header
    bgzip output/eh_output/"${sample_id}".vcf
    tabix -p vcf output/eh_output/"${sample_id}".vcf.gz
    ~/bcftools/bcftools sort -Oz output/eh_output/"${sample_id}".vcf.gz -o output/eh_output/"${sample_id}"_sorted.vcf.gz
    tabix -p vcf output/eh_output/"${sample_id}"_sorted.vcf.gz

done

In [ ]:
 
# merging sample VCFs into population VCFs

populations=("africa" "europe" "east_asia" "south_asia" "america")

for population in "${populations[@]}"; do
        
    # comma-seperated list of vcfs
    sample_vcfs=$(ls -1 output/eh_output/*_sorted.vcf.gz | \
    grep -f str_resources/"${population}"_sample_ids.txt | tr "\n" "," | sed 's/,$//')
    
    mergeSTR \
        --vcfs $sample_vcfs \
        --vcftype 'eh' \
        --out output/eh_output/"${population}"_eh
        
    bgzip output/eh_output/"${population}"_eh.vcf
    tabix -p vcf output/eh_output/"${population}"_eh.vcf.gz
    
    # filtering with dumpSTR
#     dumpSTR --vcf output/eh_output/"${population}"_eh.vcf \
#         --out output/eh_output/"${population}"_eh_filt \
#         --vcftype 'eh' 
        
#     bgzip output/eh_output/"${population}"_eh_filt.vcf
#     tabix -p vcf output/eh_output/"${population}"_eh_filt.vcf.gz

done


## GangSTR

In [ ]:
 
mamba activate gangstr

reference_genome="references/reference_chroms.fa"
output_dir=output/gangstr_output
male_list=str_resources/bams_male.txt
female_list=str_resources/bams_female.txt
gangstr_reference_strs=str_resources/gangstr_reference.bed

# male samples
for sample_bam in $(cat $male_list); do

    sample_id=$(basename $sample_bam | cut -f1 -d".")

    GangSTR --bam $sample_bam \
            --bam-samps $sample_id \
            --regions $gangstr_reference_strs \
            --samp-sex M \
            --ref $reference_genome \
            --out $output_dir/$sample_id \
            --insertmean 1 \
            --insertsdev 1 \
            --minmatch 1 \
            --min-sample-reads 1
            
    ~/bcftools/bcftools sort -Oz output/gangstr_output/"${sample_id}".vcf -o output/gangstr_output/"${sample_id}"_sorted.vcf.gz
    tabix -p vcf output/gangstr_output/"${sample_id}"_sorted.vcf.gz

done


#female samples
for sample_bam in $(cat $female_list); do

    sample_id=$(basename $sample_bam | cut -f1 -d".")

    GangSTR --bam $sample_bam \
            --bam-samps $sample_id \
            --regions $gangstr_reference_strs \
            --samp-sex F \
            --ref $reference_genome \
            --out $output_dir/$sample_id \
            --insertmean 1 \
            --insertsdev 1 \
            --minmatch 1 \
            --min-sample-reads 1

    ~/bcftools/bcftools sort -Oz output/gangstr_output/"${sample_id}".vcf -o output/gangstr_output/"${sample_id}"_sorted.vcf.gz
    tabix -p vcf output/gangstr_output/"${sample_id}"_sorted.vcf.gz
            

done

mamba deactivate

In [ ]:
 
# merging sample VCFs to population VCF
populations=("africa" "europe" "east_asia" "south_asia" "america")

for population in "${populations[@]}"; do
        
    sample_vcfs=$(ls -1 output/gangstr_output/*_sorted.vcf.gz | \
    grep -f str_resources/"${population}"_sample_ids.txt | tr "\n" "," | sed 's/,$//')
    
    mergeSTR \
        --vcfs $sample_vcfs \
        --vcftype 'gangstr' \
        --out output/gangstr_output/"${population}"_gangstr
        
        
    bgzip output/gangstr_output/"${population}"_gangstr.vcf
    tabix -p vcf output/gangstr_output/"${population}"_gangstr.vcf.gz
    

    
#     # filtering with dumpSTR
#     dumpSTR --vcf output/gangstr_output/"${population}"_gangstr.vcf \
#         --out output/gangstr_output/"${population}"_gangstr_filt \
#         --vcftype 'gangstr' 
        
#     bgzip output/gangstr_output/"${population}"_gangstr_filt.vcf
#     tabix -p vcf output/gangstr_output/"${population}"_gangstr_filt.vcf.gz

done

## adVNTR

In [ ]:
 
mamba activate advntr

advntr_model=str_resources/vntr_data/hg19_selected_VNTRs_Illumina.db
output_dir=output/advntr_output
advntr_dir=output/advntr_output/advntr_dir
bam_list=str_resources/bam_list.txt


# main script
for sample_bam in $(cat $bam_list); do

    sample_id=$(basename $sample_bam | cut -f1 -d".")

    advntr genotype \
            --vntr_id 301645 \
            --alignment_file $sample_bam \
            --outfmt vcf \
            --outfile $output_dir/"${sample_id}".vcf \
            --models $advntr_model \
            --working_directory $advntr_dir
    
    bgzip $output_dir/"${sample_id}".vcf
    tabix output/advntr_output/"${sample_id}".vcf.gz
    
    ~/bcftools/bcftools sort -Oz output/advntr_output/"${sample_id}".vcf.gz -o output/advntr_output/"${sample_id}"_sorted.vcf.gz
    tabix -p vcf output/advntr_output/"${sample_id}"_sorted.vcf.gz

done

mamba deactivate

In [ ]:
 
populations=("africa" "europe" "east_asia" "south_asia" "america")

for population in "${populations[@]}"; do
        
    # comma-seperated list of vcfs
    sample_vcfs=$(ls -1 output/advntr_output/*_sorted.vcf.gz | \
    grep -f str_resources/"${population}"_sample_ids.txt | tr "\n" "," | sed 's/,$//')
    
    mergeSTR \
        --vcfs $sample_vcfs \
        --vcftype 'advntr' \
        --out output/advntr_output/"${population}"_advntr
  
    bgzip output/advntr_output/"${population}"_advntr.vcf
    tabix -p vcf output/advntr_output/"${population}"_advntr.vcf.gz
  
    # filtering with dumpSTR
#     dumpSTR --vcf output/advntr_output/"${population}"_advntr.vcf \
#         --out output/advntr_output/"${population}"_advntr_filt \
#         --vcftype 'advntr' 
    
#     bgzip output/advntr_output/"${population}"_advntr_filt.vcf
#     tabix -p vcf output/advntr_output/"${population}"_advntr_filt.vcf.gz

done

## Merging population VCFs from each tools

In [ ]:
 
# gangstr
mergeSTR \
    --vcfs output/gangstr_output/africa_gangstr.vcf.gz,output/gangstr_output/east_asia_gangstr.vcf.gz,output/gangstr_output/south_asia_gangstr.vcf.gz,output/gangstr_output/america_gangstr.vcf.gz,output/gangstr_output/europe_gangstr.vcf.gz \
    --vcftype 'gangstr' \
    --out output/ensembletr_output/gangstr_strs

bgzip output/ensembletr_output/gangstr_strs.vcf
tabix -p vcf output/ensembletr_output/gangstr_strs.vcf.gz

# expansionhunter
mergeSTR \
    --vcfs output/eh_output/africa_eh.vcf.gz,output/eh_output/east_asia_eh.vcf.gz,output/eh_output/south_asia_eh.vcf.gz,output/eh_output/america_eh.vcf.gz,output/eh_output/europe_eh.vcf.gz \
    --vcftype 'eh' \
    --out output/ensembletr_output/eh_strs

bgzip output/ensembletr_output/eh_strs.vcf
tabix -p vcf output/ensembletr_output/eh_strs.vcf.gz

#advntr
mergeSTR \
    --vcfs output/advntr_output/africa_advntr.vcf.gz,output/advntr_output/east_asia_advntr.vcf.gz,output/advntr_output/south_asia_advntr.vcf.gz,output/advntr_output/america_advntr.vcf.gz,output/advntr_output/europe_advntr.vcf.gz \
    --vcftype 'advntr' \
    --out output/ensembletr_output/advntr_strs 
        
bgzip output/ensembletr_output/advntr_strs.vcf
tabix -p vcf output/ensembletr_output/advntr_strs.vcf.gz

# hipstr
mergeSTR \
    --vcfs output/hipstr_output/africa_hipstr_sorted.vcf.gz,output/hipstr_output/east_asia_hipstr_sorted.vcf.gz,output/hipstr_output/south_asia_hipstr_sorted.vcf.gz,output/hipstr_output/america_hipstr_sorted.vcf.gz,output/hipstr_output/europe_hipstr_sorted.vcf.gz \
    --vcftype 'hipstr' \
    --out output/ensembletr_output/hipstr_strs

bgzip output/ensembletr_output/hipstr_strs.vcf
tabix -p vcf output/ensembletr_output/hipstr_strs.vcf.gz

## EnsembleTR consensus genotyping

In [ ]:
reference_genome="references/reference_chroms.fa"

EnsembleTR \
    --ref $reference_genome \
    --vcfs output/ensembletr_output/advntr_strs.vcf.gz,output/ensembletr_output/eh_strs.vcf.gz,output/ensembletr_output/gangstr_strs.vcf.gz,output/ensembletr_output/hipstr_strs.vcf.gz \
    --out output/ensembletr_output/ensembletr.vcf
    
bgzip output/ensembletr_output/ensembletr.vcf
tabix -p vcf output/ensembletr_output/ensembletr.vcf.gz

# ~/bcftools/bcftools sort -Oz output/ensembletr_output/ensembletr.vcf.gz -o output/ensembletr_output/ensembletr_sorted.vcf.gz
# tabix -p vcf output/ensembletr_output/ensembletr_sorted.vcf.gz